# Sensitivity analysis on previous work for 30°, 45° and 60° data

Data with
aspect angles of 30, 45 and 60 degrees respectively, training data from
5 people, testing on person A, no walking test samples.

In [1]:
# Needed to allow editing using PyCharm
%load_ext autoreload
%autoreload 2

The following cell is needed for compatibility when using both CoLab and Local Jupyter notebook.
It sets the appropriate file path for the data and also installs local packages such as models.

In [ ]:
import os
if os.getcwd() == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    FILE_PATH = '/content/gdrive/My Drive/Level-4-Project/data/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
else:
    FILE_PATH = "C:/Users/macka/Google Drive/Level-4-Project/data/"
    
from src.models.original_models import cifar_xdeg
from src.data import load_data

In [2]:
import numpy as np
import sys
from six.moves import cPickle
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
import sys
from sklearn.metrics import classification_report,confusion_matrix
import csv
from keras.models import load_model

Using TensorFlow backend.


In [3]:
# Needed as originally code was for theano backend but now using tensor flow
from keras import backend as K
K.set_image_dim_ordering('th')

In [7]:
data_folders = ['cifar_30deg', 'cifar_45deg', 'cifar_60deg']

In [8]:
batch_size = 100
nb_classes = 6
nb_epoch = 100
nb_epoch = 1 # originally 100
data_augmentation = False
nb_train_samples = 34260

# input image dimensions
img_rows, img_cols = 75, 75
# the CIFAR10 images are RGB

In [9]:
for data_folder in data_folders:
    # the data, shuffled and split between train and test sets
    (X_train, y_train), (X_test, y_test) = load_data.load_data((FILE_PATH + data_folder), nb_train_samples)
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    
    model = cifar_xdeg.make_model(img_rows, img_cols, nb_classes)
    
    # let's train the model using SGD + momentum (how original).
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    if not data_augmentation:
        print('Not using data augmentation.')
        hist = model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  epochs=nb_epoch,
                  validation_data=(X_test, Y_test),
                  shuffle=True)

        y_pred = model.predict_classes(X_test)
        print(y_pred)
        target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting']
        #     sys.stdout = open('/home/aleksandar/sets/cifar_30deg/results/report_30deg.txt', "w")
        print("CLASSIFICATION REPORT:")
        print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
        print ("\n")
        print("CONFUSION MATRIX:")
        print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))

        #     with open('/home/aleksandar/sets/cifar_30deg/results/graphs_30deg.csv', "w") as f:
        #        w = csv.writer(f)
        #        train_loss = hist.history['loss']
        #        val_loss = hist.history['val_loss']
        #        train_acc = hist.history['acc']
        #        val_acc = hist.history['val_acc']
        #        helplist = list(zip(train_loss, train_acc, val_loss, val_acc))
        #        w.writerow("train_loss, train_acc, val_loss, val_acc")
        #        for val in helplist:
        #           w.writerow([val])


        #     model.save('/home/aleksandar/sets/cifar_30deg/results/model_30deg.h5')
        #-----------------------

    else:
        print('Using real-time data augmentation.')

        # this will do preprocessing and realtime data augmentation
        datagen = ImageDataGenerator(
          featurewise_center=False,  # set input mean to 0 over the dataset
          samplewise_center=False,  # set each sample mean to 0
          featurewise_std_normalization=False,  # divide inputs by std of the dataset
          samplewise_std_normalization=False,  # divide each input by its std
          zca_whitening=False,  # apply ZCA whitening
          rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
          width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
          height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
          horizontal_flip=True,  # randomly flip images
          vertical_flip=False)  # randomly flip images

        # compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied)
        datagen.fit(X_train)

        # fit the model on the batches generated by datagen.flow()
        hist = model.fit_generator(datagen.flow(X_train, Y_train,
                          batch_size=batch_size),
                          samples_per_epoch=X_train.shape[0],
                          nb_epoch=nb_epoch,
                          validation_data=(X_test, Y_test))

        y_pred = model.predict_classes(X_test)
        print(y_pred)
        target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting']
        #     sys.stdout = open('/home/aleksandar/sets/cifar_30deg/results/report_30deg.txt', "w")
        print("CLASSIFICATION REPORT:")
        print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
        print ("\n")
        print("CONFUSION MATRIX:")
        print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))

        #     with open('/home/aleksandar/sets/cifar_30deg/results/graphs_30deg.csv', "w") as f:
        #        w = csv.writer(f)
        #        train_loss = hist.history['loss']
        #        val_loss = hist.history['val_loss']
        #        train_acc = hist.history['acc']
        #        val_acc = hist.history['val_acc']
        #        helplist = list(zip(train_loss, train_acc, val_loss, val_acc))
        #        w.writerow("train_loss, train_acc, val_loss, val_acc")
        #        for val in helplist:
        #           w.writerow([val])


        #     model.save('/home/aleksandar/sets/cifar_30deg/results/model_30deg.h5')

X_train shape: (34260, 1, 75, 75)
34260 train samples
6852 test samples
Not using data augmentation.
Train on 34260 samples, validate on 6852 samples
Epoch 1/1
34260/34260 [==============================] - 160s 5ms/step - loss: 0.6041 - acc: 0.7577 - val_loss: 0.2715 - val_acc: 0.9067
[4 2 1 ... 3 4 4]
CLASSIFICATION REPORT:
                  precision    recall  f1-score   support

ArmFasterTowards       0.90      0.77      0.83      1142
ArmSlowerTowards       0.97      0.97      0.97      1142
     CirclingArm       1.00      0.85      0.92      1142
        Clapping       1.00      1.00      1.00      1142
       PickingUp       0.75      0.84      0.79      1142
         Sitting       0.86      1.00      0.93      1142

       micro avg       0.91      0.91      0.91      6852
       macro avg       0.91      0.91      0.91      6852
    weighted avg       0.91      0.91      0.91      6852



CONFUSION MATRIX:
[[ 884   22    0    0  232    4]
 [  30 1110    0    0    2    0]
 [ 